c:\Users\91790\anaconda3\Lib\site-packages\transformers\utils\generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Time taken to import: 5.41 seconds


c:\Users\91790\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\91790\anaconda3\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\91790\anaconda3\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
c:\Users\91790\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. I

Time taken to load model and tokenizer: 4.27 seconds


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [ ]:
import time 
s = time.time()
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import torch.nn.functional as F
e = time.time()
print(f"Time taken to import: {e - s:.2f} seconds")

# Load the model and tokenizer
f = time.time()
model_name = "naver/splade-cocondenser-ensembledistil"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)
g = time.time()
print(f"Time taken to load model and tokenizer: {g - f:.2f} seconds")
model.eval()


def splade_encode(texts):
    with torch.no_grad():
        inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
        out = model(**inputs)
        # Apply ReLU and LogSumExp pooling (as used in SPLADE)
        scores = F.relu(out.logits) * inputs.attention_mask.unsqueeze(-1)
        scores = torch.log(1 + torch.sum(scores, dim=1))  # LogSumExp approx
        return scores

docs = [
    "Summarize the following article into a concise paragraph that highlights the key points, main arguments, and conclusions. Focus on preserving the factual accuracy and logical flow, and keep the summary within 100 words. Ensure technical or domain-specific terms are retained when relevant to context.",
    "Generate Python code that implements the following functionality: a function that accepts a list of integers and returns a new list containing only the even numbers, sorted in ascending order. The code should be clean, readable, and follow PEP8 conventions. Avoid using external libraries unless necessary.",
    "Generate 5 advanced-level interview questions for a candidate applying for a machine learning engineering role. The questions should test the candidate's understanding of ML theory, model optimization, production deployment, and performance evaluation. Avoid basic questions — target real-world expertise and problem-solving.",
    "Translate the following technical paragraph from English to German. Maintain accuracy, terminology, and formatting, especially for code or formula-like elements. Ensure that the translation reflects native-level fluency and reads naturally in the target language. Avoid literal translations that distort the meaning.",
    "Write a compelling product description for a new AI-powered note-taking app that automatically summarizes meetings, organizes tasks, and syncs with calendars. Highlight its key features, benefits, and how it saves time for professionals. The tone should be engaging, concise, and persuasive for tech-savvy users."
]

queries = ["translation prompt"]

doc_vecs = splade_encode(docs)  # [num_docs x vocab_size]
query_vecs = splade_encode(queries)

# Dot-product scoring (sparse vector math)
scores = torch.matmul(query_vecs, doc_vecs.T)
topk = torch.topk(scores, k=2)

print("Top results:", [docs[i] for i in topk.indices[0]])

Top results: ['Translate the following technical paragraph from English to German. Maintain accuracy, terminology, and formatting, especially for code or formula-like elements. Ensure that the translation reflects native-level fluency and reads naturally in the target language. Avoid literal translations that distort the meaning.', 'Summarize the following article into a concise paragraph that highlights the key points, main arguments, and conclusions. Focus on preserving the factual accuracy and logical flow, and keep the summary within 100 words. Ensure technical or domain-specific terms are retained when relevant to context.']
